In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import *

from pyspark.sql.window import Window

In [8]:
sc.setCheckpointDir("/dados10t/datalake/raw/checkpoint_dir/")

In [9]:
path = "/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/"

### tb location

In [4]:
#df = spark.read.csv("/home/juracy/codigos_2021/fiocruz/omop/tabela_para_insert/tb_location.csv",sep = ",",header=True,inferSchema=True)
df = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_location.parquet/")

In [5]:
df.count()

5570

In [6]:
df.select("location_id").distinct().count()

5570

In [7]:
df.printSchema()

root
 |-- location_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- location_source_value: string (nullable = true)
 |-- country_concept_id: integer (nullable = true)
 |-- county: string (nullable = true)
 |-- country_source_value: string (nullable = true)



In [8]:
df.show(3)

+-----------+-------------------+-----+---------------------+------------------+-------+--------------------+
|location_id|               city|state|location_source_value|country_concept_id| county|country_source_value|
+-----------+-------------------+-----+---------------------+------------------+-------+--------------------+
|    5200050|    Abadia de Goiás|   GO|              5200050|          41892130|    sul|              Brazil|
|    3100104|Abadia dos Dourados|   MG|              3100104|          41892130|Sudeste|              Brazil|
|    5200100|          Abadiânia|   GO|              5200100|          41892130|    sul|              Brazil|
+-----------+-------------------+-----+---------------------+------------------+-------+--------------------+
only showing top 3 rows



In [9]:

options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.location",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

df.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### tb care site

In [4]:
#df = spark.read.csv("/home/juracy/codigos_2021/fiocruz/omop/tabela_para_insert/tb_location.csv",sep = ",",header=True,inferSchema=True)
care = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_care_site.parquet/")

In [5]:
care.count()

482479

In [6]:
care.select("care_site_id").distinct().count()

482479

In [7]:
care.orderBy(F.asc("care_site_id")).show(100)

+------------+--------------------+---------------------------+-----------+----------------------+-----------------------------+
|care_site_id|      care_site_name|place_of_service_concept_id|location_id|care_site_source_value|place_of_service_source_value|
+------------+--------------------+---------------------------+-----------+----------------------+-----------------------------+
|     0000019|POLICLINICA DR JA...|                   38004247|    2602902|               0000019|                  POLICLINICA|
|     0000027|CASA DE SAUDE SAN...|                   38004515|    2602902|               0000027|               HOSPITAL GERAL|
|     0000035|HOSPITAL MENDO SA...|                   38004515|    2602902|               0000035|               HOSPITAL GERAL|
|     0000043|POLICLINICA DR MA...|                   38004247|    2602902|               0000043|                  POLICLINICA|
|     0000051|POLICLINICA VICEN...|                   38004247|    2602902|               0000051

In [12]:
care = care.withColumn("care_site_id",F.col("care_site_id").cast(IntegerType()))
care = care.withColumn("location_id",F.col("location_id").cast(IntegerType()))
care = care.withColumn("place_of_service_concept_id",F.col("place_of_service_concept_id").cast(IntegerType()))
care = care.withColumn("place_of_service_source_value", F.substring("place_of_service_source_value",1,50))

In [13]:
care.printSchema()

root
 |-- care_site_id: integer (nullable = true)
 |-- care_site_name: string (nullable = true)
 |-- place_of_service_concept_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- care_site_source_value: string (nullable = true)
 |-- place_of_service_source_value: string (nullable = true)



In [14]:
care.show(10,False)

+------------+----------------------------------------------------------+---------------------------+-----------+----------------------+--------------------------------------------------+
|care_site_id|care_site_name                                            |place_of_service_concept_id|location_id|care_site_source_value|place_of_service_source_value                     |
+------------+----------------------------------------------------------+---------------------------+-----------+----------------------+--------------------------------------------------+
|6290434     |LEONARDO RODRIGUES RESENDE                                |38004207                   |5002704    |6290434               |CONSULTORIO ISOLADO                               |
|6280250     |LABORATORIO SANTO ANTONIO                                 |38004678                   |5005004    |6280250               |UNIDADE DE APOIO DIAGNOSE E TERAPIA (SADT ISOLADO)|
|6284744     |CONSULTORIO DE NATUROLOGIA                    

In [15]:

options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.care_site",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

care.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### tb paciente

In [10]:
pacientes = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_person.parquet/")

In [11]:
pacientes.count()

188537327

In [12]:
pacientes.describe("person_id").show()

+-------+--------------------+
|summary|           person_id|
+-------+--------------------+
|  count|           188537327|
|   mean|         9.4268663E7|
| stddev|5.4426038392208815E7|
|    min|                   0|
|    max|           188537326|
+-------+--------------------+



In [13]:
pacientes.select("person_id").distinct().count()

188537327

In [14]:
pacientes.printSchema()

root
 |-- raca: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: string (nullable = true)
 |-- race_concept_id: string (nullable = true)
 |-- year_of_birth: long (nullable = true)
 |-- month_of_birth: long (nullable = true)
 |-- day_of_birth: long (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)



In [16]:
pacientes_v2 = pacientes.withColumn("year_of_birth",F.col("year_of_birth").cast(IntegerType()))
pacientes_v2 = pacientes_v2.withColumn("month_of_birth",F.col("month_of_birth").cast(IntegerType()))
pacientes_v2 = pacientes_v2.withColumn("day_of_birth",F.col("day_of_birth").cast(IntegerType()))
pacientes_v2 = pacientes_v2.withColumn("ethnicity_concept_id",F.col("ethnicity_concept_id").cast(IntegerType()))
pacientes_v2 = pacientes_v2.withColumn("gender_concept_id",F.col("gender_concept_id").cast(IntegerType()))
pacientes_v2 = pacientes_v2.withColumn("race_concept_id",F.col("race_concept_id").cast(IntegerType()))

In [17]:
pacientes_v2 = pacientes_v2.drop(*['raca','sexo','id_vigvac'])

In [16]:
#pacientes_v2.show(5)

In [18]:
pacientes_v2.printSchema()

root
 |-- gender_source_value: string (nullable = true)
 |-- race_source_value: string (nullable = true)
 |-- gender_concept_id: integer (nullable = true)
 |-- race_concept_id: integer (nullable = true)
 |-- year_of_birth: integer (nullable = true)
 |-- month_of_birth: integer (nullable = true)
 |-- day_of_birth: integer (nullable = true)
 |-- birth_datetime: timestamp (nullable = true)
 |-- ethnicity_concept_id: integer (nullable = true)
 |-- person_id: long (nullable = true)



In [19]:
pacientes_v2.groupBy("gender_concept_id").count().orderBy(F.desc("count")).show(500,False) 

+-----------------+--------+
|gender_concept_id|count   |
+-----------------+--------+
|8532             |97691350|
|8507             |90840521|
|null             |5456    |
+-----------------+--------+



In [20]:
pacientes_v2 = pacientes_v2.withColumn("race_concept_id",F.when(F.col("race_concept_id").isNull(),0).otherwise(F.col("race_concept_id")) )

In [20]:
#pacientes_v2 = pacientes_v2.withColumn("year_of_birth",F.when(F.col("year_of_birth").isNull(),0).otherwise(F.col("year_of_birth")) )

In [21]:
pacientes_v3 = pacientes_v2.filter(F.col("gender_concept_id").isNotNull())

In [22]:
pacientes_v3.count()

188531871

In [23]:
pacientes_v3.filter(F.col("year_of_birth").isNotNull()).count()

187639193

In [24]:
pacientes_v3.filter(F.col("year_of_birth").isNull()).count()

892678

In [25]:
pacientes_v4 = pacientes_v3.filter(F.col("year_of_birth").isNotNull() ).checkpoint(eager=True)

In [26]:
pacientes_v4.filter(F.col("birth_datetime") <= "1900-01-01").count()

4683

In [27]:
pacientes_v5 = pacientes_v4.filter(F.col("birth_datetime") >= "1900-01-01")

In [28]:
pacientes_v5.show(10)

+-------------------+-----------------+-----------------+---------------+-------------+--------------+------------+-------------------+--------------------+---------+
|gender_source_value|race_source_value|gender_concept_id|race_concept_id|year_of_birth|month_of_birth|day_of_birth|     birth_datetime|ethnicity_concept_id|person_id|
+-------------------+-----------------+-----------------+---------------+-------------+--------------+------------+-------------------+--------------------+---------+
|          Masculino|           BRANCA|             8507|           8527|         2012|            10|          20|2012-10-20 00:00:00|            38003563| 11328339|
|           Feminino|           BRANCA|             8532|           8527|         2015|            11|          16|2015-11-16 00:00:00|            38003563| 11328340|
|           Feminino|           BRANCA|             8532|           8527|         1965|            12|          20|1965-12-20 00:00:00|            38003563| 11328341

In [30]:
pacientes_v5.count()

187634532

In [32]:
#17:50

In [31]:

options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.person",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

pacientes_v5.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

In [33]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
print("now =", now)

now = 2023-10-18 18:18:43.710524


### tb periodo observação

In [36]:
peri_obs = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_observation_period.parquet")

In [37]:
peri_obs.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: string (nullable = true)
 |-- person_id: long (nullable = true)
 |-- observation_period_id: long (nullable = true)



In [38]:
peri_obs.count()

188537327

In [39]:
peri_obs_v2 = peri_obs.withColumn("period_type_concept_id",F.col("period_type_concept_id").cast(IntegerType()))

In [40]:
peri_obs_v2.describe("person_id").show()

+-------+-------------------+
|summary|          person_id|
+-------+-------------------+
|  count|          188537327|
|   mean|        9.4268663E7|
| stddev|5.442603839220903E7|
|    min|                  0|
|    max|          188537326|
+-------+-------------------+



In [41]:
peri_obs_v2.describe("observation_period_id").show()

+-------+---------------------+
|summary|observation_period_id|
+-------+---------------------+
|  count|            188537327|
|   mean|  9.426866299999999E7|
| stddev|  5.442603839220844E7|
|    min|                    0|
|    max|            188537326|
+-------+---------------------+



In [43]:
peri_obs_v3 = peri_obs_v2.drop(*['id_vigvac'])

In [44]:
peri_obs_v3.printSchema()

root
 |-- observation_period_start_date: date (nullable = true)
 |-- observation_period_end_date: date (nullable = true)
 |-- period_type_concept_id: integer (nullable = true)
 |-- person_id: long (nullable = true)
 |-- observation_period_id: long (nullable = true)



In [46]:
#19:24

In [45]:

options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.observation_period",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

peri_obs_v3.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

In [47]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
print("now =", now)

now = 2023-10-18 19:51:08.760588


### tb drug exposure

In [1]:
drug = spark.read.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_drug_exposure_v6.parquet/")

In [2]:
drug.show(10)

+------------------------+--------------------+--------------------------+--------------------+---------------+---------+-------------------+----------------+
|drug_exposure_start_date|   drug_source_value|drug_exposure_end_datetime|drug_type_concept_id|drug_concept_id|person_id|visit_occurrence_id|drug_exposure_id|
+------------------------+--------------------+--------------------------+--------------------+---------------+---------+-------------------+----------------+
|              2022-04-28|Vacina covid-19 -...|                2022-04-28|               32818|       37003436|  9441615|          321953373|       410442167|
|              2021-09-09|Covid-19-AstraZeneca|                2021-09-09|               32818|         724905|  9441657|          321953467|       410442168|
|              2021-04-17|Covid-19-AstraZeneca|                2021-04-17|               32818|         724905|  9441668|          321953499|       410442169|
|              2021-07-14|Vacina covid-19 -...

In [3]:
drug.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: string (nullable = true)
 |-- drug_concept_id: long (nullable = true)
 |-- person_id: long (nullable = true)
 |-- visit_occurrence_id: long (nullable = true)
 |-- drug_exposure_id: integer (nullable = true)



In [4]:
drug.count() #446124123

446134022

In [5]:
drug.select("drug_exposure_id").distinct().count()

446134022

In [6]:
drug.select("visit_occurrence_id").distinct().count()

446134022

In [7]:
drug.select("person_id").distinct().count()

183734003

In [8]:
drug = drug.withColumn("person_id",F.col("person_id").cast(IntegerType()))
drug = drug.withColumn("visit_occurrence_id",F.col("visit_occurrence_id").cast(IntegerType()))

In [9]:
drug.select("visit_occurrence_id").distinct().count()

446134022

In [10]:
drug.select("person_id").distinct().count()

183734003

In [11]:
drug = drug.withColumn("drug_type_concept_id",F.col("drug_type_concept_id").cast(IntegerType()))

In [12]:
drug.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_datetime: date (nullable = true)
 |-- drug_type_concept_id: integer (nullable = true)
 |-- drug_concept_id: long (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)
 |-- drug_exposure_id: integer (nullable = true)



In [13]:
drug.describe("visit_occurrence_id").show()

+-------+--------------------+
|summary| visit_occurrence_id|
+-------+--------------------+
|  count|           446134022|
|   mean|3.0849558567280805E8|
| stddev|1.2881189733768925E8|
|    min|            85386715|
|    max|           531604067|
+-------+--------------------+



In [14]:
drug.describe("drug_exposure_id").show()

+-------+--------------------+
|summary|    drug_exposure_id|
+-------+--------------------+
|  count|           446134022|
|   mean|2.2306701050000003E8|
| stddev| 1.287877989925135E8|
|    min|                   0|
|    max|           446134021|
+-------+--------------------+



In [15]:
len("Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag")

45

In [16]:
drug.groupBy("drug_concept_id","drug_source_value").count().orderBy(F.desc("drug_concept_id")).show(30,False)

+---------------+---------------------------------------------------------+---------+
|drug_concept_id|drug_source_value                                        |count    |
+---------------+---------------------------------------------------------+---------+
|37003436       |Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer|181277235|
|36941610       |Covid-19-Coronavac-Sinovac/Butantan                      |103615299|
|739906         |Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag            |25182248 |
|724905         |Covid-19-AstraZeneca                                     |136059240|
+---------------+---------------------------------------------------------+---------+



In [17]:
drug = drug.withColumn("drug_source_value",F.when(F.col("drug_source_value") == "Vacina covid-19 - BNT162b2 - BioNTech/Fosun Pharma/Pfizer", "pfizer" ).otherwise(F.col("drug_source_value"))  )

In [30]:
#drug = drug.withColumn("drug_concept_id",F.when(F.col("drug_concept_id") == "724905", "pfizer" ).otherwise(F.col("drug_source_value"))  )

In [31]:
#drug.groupBy("drug_concept_id","drug_source_value").count().orderBy(F.desc("drug_concept_id")).show(30,False)

In [32]:
#drug_v2 = drug.filter(F.col("drug_concept_id") != "724905")

In [18]:
drug.filter(F.col("drug_exposure_end_datetime").isNull()  ).count()

0

In [19]:
drug = drug.withColumnRenamed("drug_exposure_end_datetime","drug_exposure_end_date")

In [21]:
drug.groupBy("drug_concept_id","drug_source_value").count().orderBy(F.desc("drug_concept_id")).show(30,False)

+---------------+---------------------------------------------+---------+
|drug_concept_id|drug_source_value                            |count    |
+---------------+---------------------------------------------+---------+
|37003436       |pfizer                                       |181277235|
|36941610       |Covid-19-Coronavac-Sinovac/Butantan          |103615299|
|739906         |Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag|25182248 |
|724905         |Covid-19-AstraZeneca                         |136059240|
+---------------+---------------------------------------------+---------+



In [22]:
drug.printSchema()

root
 |-- drug_exposure_start_date: date (nullable = true)
 |-- drug_source_value: string (nullable = true)
 |-- drug_exposure_end_date: date (nullable = true)
 |-- drug_type_concept_id: integer (nullable = true)
 |-- drug_concept_id: long (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- visit_occurrence_id: integer (nullable = true)
 |-- drug_exposure_id: integer (nullable = true)



In [23]:

options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.drug_exposure",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

drug.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()